In [1]:
suppressPackageStartupMessages({
    library(dplyr)
    library(tidyverse)
    library(susieR)
    library(glue)
    library(latex2exp) # for latex the -log10(P)
})
library(ggplotify)

In [2]:
L = 10
DATA_DIR = "/u/project/pasaniuc/kangchen/2021-admix-corr/experiments/05-real-locus"
SUSIE_DIR = '/u/project/pasaniuc/yiding/projects/admix-corr/out'
FIG_DIR = './locus-plot'

In [3]:
df_info = data.frame(
    study = character(),
    trait = character(),
    chrom = integer(),
    pos = integer()
)

df_info <- df_info %>% add_row(study = "page", trait = "crp", chrom = 1, pos = 159638387)
df_info <- df_info %>% add_row(study = "page", trait = "hdl", chrom = 11, pos = 116810245)
df_info <- df_info %>% add_row(study = "ukb", trait = "erythrocyte", chrom = 16, pos = 357598)
df_info <- df_info %>% add_row(study = "ukb", trait = "log_triglycerides", chrom = 19, pos = 35065736)
df_info <- df_info %>% add_row(study = "ukb", trait = "MCH", chrom = 16, pos = 1121191)


# summarize susie

In [23]:
fnlist <- function(x, fil){ z <- deparse(substitute(x))
                         cat(z, "\n", file=fil)
                         nams=names(x) 
                   for (i in seq_along(x) ){ cat(nams[i], "\t",  x[[i]], "\n", 
                                            file=fil, append=TRUE) }
                         }
fnlist(cs, "test.txt")

In [24]:
summary_cs = data.frame(
    study = character(),
    trait = character(),
    chrom = integer(),
    pos = integer(),
    nCS = integer(),
    size = character()
)

for (i in 1:5){
    study = df_info[i, 'study']
    trait = df_info[i, 'trait']
    chrom = df_info[i, 'chrom'] 
    pos = df_info[i,'pos']

    susie <- readRDS(glue('{SUSIE_DIR}/{study}-{trait}-{chrom}-{pos}/susie.L={L}.rds')) 
    X <- readRDS(glue('{SUSIE_DIR}/{study}-{trait}-{chrom}-{pos}/susie.X.rds'))

    cs <- susie_get_cs(susie, X = X, dedup = TRUE, min_abs_corr = 0.5, coverage = 0.95)$cs
    summary_cs <- summary_cs %>% 
        add_row(
            study = study,
            trait = trait, 
            chrom = chrom,
            pos = pos,
            nCS = length(cs),
            size = paste(sapply(cs, function(x) length(x)), collapse = ',')
        )
    cs$PIP <- susie_get_pip(susie)
    fnlist(cs, glue('out/locus-plot/{study}-{trait}-{chrom}-{pos}.txt'))
}

summary_cs

study,trait,chrom,pos,nCS,size
<chr>,<chr>,<dbl>,<dbl>,<int>,<chr>
page,crp,1,159638387,4,"1,9,6,5"
page,hdl,11,116810245,2,"1,1"
ukb,erythrocyte,16,357598,2,"4,30"
ukb,log_triglycerides,19,35065736,1,6
ukb,MCH,16,1121191,7,"1,1,1,3,4,41,12"


# manhattan and susie plot

In [9]:

    study = df_info[i, 'study']
    trait = df_info[i, 'trait']
    chrom = df_info[i, 'chrom'] 
    pos = df_info[i,'pos']


    # load susie object
    susie <- readRDS(glue('{SUSIE_DIR}/{study}-{trait}-{chrom}-{pos}/susie.L={L}.rds'))

    # read association
    df_assoc = read_tsv(
        glue("{DATA_DIR}/out/gwas-assoc/{study}-{trait}.assoc"), 
        col_types = cols()
    )

    # get the region for plot 
    plot_df <- df_assoc %>% 
        filter(`#CHROM` == chrom, POS > pos - 1.5e6, POS < pos + 1.5e6)
    # set the SNP index
    plot_df <- plot_df %>% 
        mutate(idx = 1:nrow(plot_df)) %>% 
        filter(P < 0.05) 

    # get clumped SNPs
    clumped_SNPs <- read_tsv(glue("{DATA_DIR}/out/gwas-assoc/{study}-{trait}.clumped"), col_types = cols()) %>% 
        filter(CHR == chrom) %>% 
        pull(SNP)
    # plot aesthetic
    plot_df <- plot_df %>%
        mutate(
            P = -log10(P), 
            ORDER = (ID %in% clumped_SNPs) * 1, # put the clumped SNPs on top of the other SNPs
            COLOR = ifelse(ORDER == 1, '#fa7f0e', alpha('#2579b4', 0.5)), 
            SIZE = ifelse(ORDER == 1, 2, 1), 
        ) %>% 
        arrange(ORDER) # put the clumped SNPs on top of the other SNPs


In [42]:

## save plot 
out_fig <- file.path(glue('{FIG_DIR}/{study}-{trait}-{chrom}-{pos}.pdf'))
pdf(file=out_fig, width = 14, height = 11)
par(mfrow = c(2, 1), mar = c(3, 7, 4, 7))


plot(x = plot_df$idx, 
     y = plot_df$P, 
     col = "blue", 
     cex = plot_df$SIZE,
     pch = 20, 
     main = glue('{study} {trait} chr{chrom}'),
     font.main = 1,
     cex.axis=2, cex.main=3, cex.lab = 2, 
     xlab = '',
     ylab = TeX('-log_{10}($P$)'))
abline(h = -log10(5/10**8), col = 'red', lwd=2, lty=2)

susie_plot(susie, 
           xlab = '',
           y = 'PIP', 
           cex.axis=2, cex.main=3, cex.lab = 2, cex = 1, col = alpha('black', 0.5))

dev.off()

png 
  2

In [7]:
for (i in 1:5){

    study = df_info[i, 'study']
    trait = df_info[i, 'trait']
    chrom = df_info[i, 'chrom'] 
    pos = df_info[i,'pos']


    # load susie object
    susie <- readRDS(glue('{SUSIE_DIR}/{study}-{trait}-{chrom}-{pos}/susie.L={L}.rds'))

    # read association
    df_assoc = read_tsv(
        glue("{DATA_DIR}/out/gwas-assoc/{study}-{trait}.assoc"), 
        col_types = cols()
    )

    # get the region for plot 
    plot_df <- df_assoc %>% 
        filter(`#CHROM` == chrom, POS > pos - 1.5e6, POS < pos + 1.5e6)
    # set the SNP index
    plot_df <- plot_df %>% 
        mutate(idx = 1:nrow(plot_df)) %>% 
        filter(P < 0.05) 

    # get clumped SNPs
    clumped_SNPs <- read_tsv(glue("{DATA_DIR}/out/gwas-assoc/{study}-{trait}.clumped"), col_types = cols()) %>% 
        filter(CHR == chrom) %>% 
        pull(SNP)
    # plot aesthetic
    plot_df <- plot_df %>%
        mutate(
            P = -log10(P), 
            ORDER = (ID %in% clumped_SNPs) * 1, # put the clumped SNPs on top of the other SNPs
            COLOR = ifelse(ORDER == 1, '#fa7f0e', alpha('#2579b4', 0.5)), 
            SIZE = ifelse(ORDER == 1, 2, 1), 
        ) %>% 
        arrange(ORDER) # put the clumped SNPs on top of the other SNPs

    ## save plot 
    out_fig <- file.path(glue('{FIG_DIR}/{study}-{trait}-{chrom}-{pos}.pdf'))
    pdf(file=out_fig, width = 14, height = 11)
    par(mfrow = c(2, 1), mar = c(3, 7, 4, 7))

    plot(x = plot_df$idx, 
         y = plot_df$P, 
         col = plot_df$COLOR, 
         cex = plot_df$SIZE,
         pch = 20, 
         main = glue('{study} {trait} chr{chrom}'),
         font.main = 1,
         cex.axis=2, cex.main=3, cex.lab = 2, 
         xlab = '',
         ylab = TeX('-log_{10}($P$)'))
    abline(h = -log10(5/10**8), col = 'red', lwd=2, lty=2)
    susie_plot(susie, 
               xlab = '',
               y = 'PIP', 
               cex.axis=2, cex.main=3, cex.lab = 2, cex = 1, col = alpha('black', 0.5))
    dev.off()
}